# RNN时间序列预测完整指南

## 教程概览

本教程全面介绍如何使用RNN进行时间序列预测，涵盖从基础到高级的多种预测策略。

### 核心内容
1. **基准模型**：简单线性回归baseline
2. **单步预测**：使用RNN预测下一个时间步
3. **深层RNN**：堆叠多层RNN提升性能
4. **多步预测**：预测未来多个时间步
5. **序列到序列**：Seq2Seq架构详解

### 预测任务类型
- **Sequence-to-One**：输入序列 → 单个值
- **Sequence-to-Sequence**：输入序列 → 输出序列
- **多步预测**：自回归预测vs一次性预测

### 学习目标
- 掌握时间序列预测的基本流程
- 理解不同RNN架构的适用场景
- 学会评估和对比模型性能
- 了解多步预测的不同策略

In [ ]:
"""
数据生成：合成时间序列
生成包含多个正弦波和噪声的时间序列数据
"""
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

# 设置随机种子确保可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

def generate_time_series(batch_size, n_steps):
    """
    生成合成的时间序列数据
    
    数据特点:
    - 由两个不同频率的正弦波叠加而成
    - 添加随机噪声增加真实性
    - 每个样本的频率和相位随机
    
    参数:
        batch_size: 生成的序列数量
        n_steps: 每个序列的时间步数
        
    返回:
        形状为 (batch_size, n_steps, 1) 的数组
    """
    # 生成随机频率和相位偏移
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    
    # 创建时间轴
    time = np.linspace(0, 1, n_steps)
    
    # 第一个正弦波：振幅0.5，频率10-20Hz
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    
    # 第二个正弦波：振幅0.2，频率20-40Hz
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    
    # 添加高斯噪声
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    
    return series[..., np.newaxis].astype(np.float32)

# 可视化几个样本
print("=" * 70)
print("生成并可视化时间序列样本")
print("=" * 70)

sample_series = generate_time_series(4, 100)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for i, ax in enumerate(axes.flat):
    ax.plot(sample_series[i, :, 0], linewidth=2)
    ax.set_title(f'时间序列样本 {i+1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('时间步', fontsize=10)
    ax.set_ylabel('值', fontsize=10)
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

print("✓ 数据生成函数定义完成")

In [ ]:
"""
准备训练数据集
任务：给定前n_steps个点，预测第n_steps+1个点（单步预测）
"""
print("\n" + "=" * 70)
print("准备数据集")
print("=" * 70)

# 生成数据
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

# 划分输入和目标
# X: 前50个时间步，y: 第51个时间步
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

print(f"训练集: X={X_train.shape}, y={y_train.shape}")
print(f"验证集: X={X_valid.shape}, y={y_valid.shape}")
print(f"测试集: X={X_test.shape}, y={y_test.shape}")
print(f"\n任务类型: Sequence-to-One (序列到单值)")
print(f"输入: {n_steps}个时间步")
print(f"输出: 1个值（下一时间步的预测）")
print("=" * 70)

## 一、基准模型：线性回归

建立一个简单的baseline模型，用于对比RNN的性能提升。

**模型架构：**
- Flatten层：将序列展平为一维向量
- Dense层：线性回归输出

**优点：** 简单快速
**缺点：** 丢失了时序信息，无法捕捉序列中的模式

In [ ]:
"""
训练基准模型
"""
print("\n" + "=" * 70)
print("基准模型：线性回归")
print("=" * 70)

# 构建模型
model_baseline = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[n_steps, 1]),
    keras.layers.Dense(1)
])

model_baseline.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
)

print("模型结构：")
model_baseline.summary()

# 训练模型（使用较少epoch进行快速测试）
print("\n开始训练...")
history_baseline = model_baseline.fit(
    X_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss, test_mae = model_baseline.evaluate(X_test, y_test, verbose=0)
print(f"\n基准模型性能:")
print(f"  测试集MSE: {test_loss:.6f}")
print(f"  测试集MAE: {test_mae:.6f}")
print("=" * 70)

## 二、单层SimpleRNN模型

使用单层SimpleRNN来捕捉时序信息。

**模型架构：**
- SimpleRNN层：50个隐藏单元，只返回最后一个时间步的输出
- Dense层：将RNN输出映射到最终预测值

**优势：**
- 能够捕捉序列中的时间依赖关系
- 参数量相对较少
- 训练速度快

In [ ]:
"""
训练单层SimpleRNN模型
"""
print("\n" + "=" * 70)
print("单层SimpleRNN模型")
print("=" * 70)

# 构建模型
model_rnn = keras.models.Sequential([
    # SimpleRNN层：处理序列数据
    # return_sequences=False（默认）：只返回最后一个时间步
    keras.layers.SimpleRNN(50, input_shape=[n_steps, 1]),
    
    # 输出层
    keras.layers.Dense(1)
])

model_rnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("模型结构：")
model_rnn.summary()

# 训练模型
print("\n开始训练...")
history_rnn = model_rnn.fit(
    X_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss_rnn, test_mae_rnn = model_rnn.evaluate(X_test, y_test, verbose=0)
print(f"\nSimpleRNN模型性能:")
print(f"  测试集MSE: {test_loss_rnn:.6f}")
print(f"  测试集MAE: {test_mae_rnn:.6f}")

# 对比基准模型
improvement = (test_loss - test_loss_rnn) / test_loss * 100
print(f"\n相比基准模型，MSE改进: {improvement:.2f}%")
print("=" * 70)

## 三、深层RNN模型

通过堆叠多层RNN来增加模型的表达能力。

**模型架构：**
- 第1层SimpleRNN：50单元，return_sequences=True
- 第2层SimpleRNN：50单元，return_sequences=True
- 第3层SimpleRNN：20单元，return_sequences=False
- Dense输出层

**设计要点：**
1. 前面的RNN层必须设置`return_sequences=True`才能将序列传递给下一层
2. 最后一层RNN只需返回最后时间步的输出
3. 层数递减可以逐步提取更抽象的特征

In [ ]:
"""
训练深层RNN模型
"""
print("\n" + "=" * 70)
print("深层RNN模型（3层堆叠）")
print("=" * 70)

model_deep = keras.models.Sequential([
    # 第1层：返回完整序列
    keras.layers.SimpleRNN(50, return_sequences=True, input_shape=[n_steps, 1]),
    
    # 第2层：继续返回序列
    keras.layers.SimpleRNN(50, return_sequences=True),
    
    # 第3层：只返回最后一个时间步
    keras.layers.SimpleRNN(20),
    
    # 输出层
    keras.layers.Dense(1)
])

model_deep.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("模型结构：")
model_deep.summary()

# 训练模型
print("\n开始训练...")
history_deep = model_deep.fit(
    X_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss_deep, test_mae_deep = model_deep.evaluate(X_test, y_test, verbose=0)
print(f"\n深层RNN模型性能:")
print(f"  测试集MSE: {test_loss_deep:.6f}")
print(f"  测试集MAE: {test_mae_deep:.6f}")

print(f"\n模型对比:")
print(f"  基准模型MSE: {test_loss:.6f}")
print(f"  单层RNN MSE: {test_loss_rnn:.6f}")
print(f"  深层RNN MSE: {test_loss_deep:.6f}")
print("=" * 70)

## 四、多步预测策略

预测未来多个时间步有两种主要策略：

### 1. 自回归预测(Autoregressive)
- 一次预测一个时间步
- 将预测值作为下一次的输入
- 优点：使用单输出模型
- 缺点：误差会累积

### 2. 序列到序列(Sequence-to-Sequence)
- 一次性预测所有未来时间步
- 使用TimeDistributed层
- 优点：避免误差累积
- 缺点：模型更复杂

下面演示自回归预测方法：

In [ ]:
"""
自回归多步预测示例
使用已训练的RNN模型预测未来10个时间步
"""
print("\n" + "=" * 70)
print("多步预测：自回归方法")
print("=" * 70)

# 生成一个测试序列
n_predict = 10
test_series = generate_time_series(1, n_steps + n_predict)
X_test_seq = test_series[:, :n_steps]
y_true = test_series[:, n_steps:]

# 自回归预测
X_current = X_test_seq.copy()
predictions = []

print(f"输入序列长度: {n_steps}")
print(f"预测步数: {n_predict}")
print("\n开始逐步预测...")

for step in range(n_predict):
    # 预测下一个时间步
    y_pred = model_rnn.predict(X_current, verbose=0)
    predictions.append(y_pred[0, 0])
    
    # 将预测值添加到输入序列中
    X_current = np.concatenate([X_current, y_pred.reshape(1, 1, 1)], axis=1)
    
    # 移除最旧的时间步，保持固定长度（滑动窗口）
    X_current = X_current[:, 1:, :]

predictions = np.array(predictions)

# 计算误差
mse = np.mean((y_true[0, :, 0] - predictions) ** 2)
print(f"\n多步预测MSE: {mse:.6f}")

# 可视化
plt.figure(figsize=(14, 5))

# 绘制输入序列
plt.plot(range(n_steps), X_test_seq[0, :, 0], 
         'b-', linewidth=2, label='输入序列')

# 绘制真实未来值
plt.plot(range(n_steps, n_steps + n_predict), y_true[0, :, 0], 
         'g-o', linewidth=2, markersize=6, label='真实值')

# 绘制预测值
plt.plot(range(n_steps, n_steps + n_predict), predictions, 
         'r--s', linewidth=2, markersize=6, label='预测值')

plt.axvline(x=n_steps, color='gray', linestyle=':', linewidth=2, alpha=0.5)
plt.text(n_steps, plt.ylim()[1]*0.9, '预测起点', 
         ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat'))

plt.title('自回归多步预测', fontsize=14, fontweight='bold')
plt.xlabel('时间步', fontsize=12)
plt.ylabel('值', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("=" * 70)

## 五、序列到序列模型(Seq2Seq)

Seq2Seq模型可以一次性预测多个未来时间步，避免自回归方法的误差累积。

**模型架构：**
- 多层SimpleRNN，全部设置`return_sequences=True`
- TimeDistributed层：对每个时间步应用相同的Dense层
- 输出：与输入相同长度的序列

**应用场景：**
- 机器翻译
- 视频帧预测
- 语音合成
- 多步时间序列预测

In [ ]:
"""
构建并训练Seq2Seq模型
"""
print("\n" + "=" * 70)
print("Seq2Seq模型：序列到序列预测")
print("=" * 70)

# 准备Seq2Seq数据
# 输入：前n_steps个点，输出：后10个点的序列
n_output_steps = 10
series_seq2seq = generate_time_series(10000, n_steps + n_output_steps)
X_train_seq = series_seq2seq[:7000, :n_steps]
y_train_seq = series_seq2seq[:7000, -n_output_steps:, np.newaxis]
X_valid_seq = series_seq2seq[7000:9000, :n_steps]
y_valid_seq = series_seq2seq[7000:9000, -n_output_steps:, np.newaxis]

print(f"训练数据形状: X={X_train_seq.shape}, y={y_train_seq.shape}")

# 构建模型
model_seq2seq = keras.models.Sequential([
    # 第1层RNN：保持序列输出
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    
    # 第2层RNN：继续保持序列
    keras.layers.SimpleRNN(20, return_sequences=True),
    
    # TimeDistributed：对每个时间步应用Dense层
    # 输出形状：(batch_size, n_output_steps, 1)
    keras.layers.TimeDistributed(keras.layers.Dense(1))
])

model_seq2seq.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n模型结构：")
model_seq2seq.summary()

# 训练（注意：y现在是序列形状，不是单值）
print("\n开始训练...")
history_seq2seq = model_seq2seq.fit(
    X_train_seq, y_train_seq,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid_seq, y_valid_seq),
    verbose=1
)

print("\n✓ Seq2Seq模型训练完成")
print("=" * 70)

## 总结与最佳实践

### 模型选择指南

| 模型类型 | 适用场景 | 优点 | 缺点 |
|---------|---------|------|------|
| 线性回归 | 简单基准 | 快速、易解释 | 无法捕捉时序模式 |
| 单层RNN | 短序列预测 | 参数少、训练快 | 长期依赖能力弱 |
| 深层RNN | 复杂时序模式 | 表达能力强 | 易过拟合、训练慢 |
| LSTM/GRU | 长序列、长期依赖 | 解决梯度消失 | 计算开销大 |
| Seq2Seq | 多步预测 | 避免误差累积 | 模型复杂度高 |

### 关键技术点

1. **return_sequences参数**
   - False（默认）：只返回最后时间步，用于Seq2One
   - True：返回所有时间步，用于堆叠RNN或Seq2Seq

2. **多步预测策略**
   - 自回归：简单但误差累积
   - Seq2Seq：复杂但更准确

3. **数据准备**
   - 归一化：提升训练稳定性
   - 滑动窗口：构造监督学习样本
   - 时间步长选择：权衡信息量和计算成本

### 性能优化建议

- 使用LSTM/GRU替代SimpleRNN
- 添加Dropout防止过拟合
- 使用双向RNN捕捉双向依赖
- 结合注意力机制提升长序列性能
- 考虑使用Transformer处理超长序列